For something sensitive and when deeling with crypto it important to use the secrets module better than the random module 

In [61]:
from collections import namedtuple
import secrets 
from typing import List,Tuple 
import numpy as np
from crypto import RandomPrime, xgcd, LCM,InverseMod

In [86]:
PrivateKey = namedtuple("Private_key", ["lam",'mu'])
PublicKey = namedtuple("Public_key",["g","n",'n_sqr'])

In [332]:
DEFAULT_BIT_SIZE = 64

In [199]:
def L(x:int,n:int) -> int:
    return (x-1)//n

In [243]:
def create_key_paire(size_bits:int = DEFAULT_BIT_SIZE): 
    p = RandomPrime(size_bits,60)
    q = RandomPrime(size_bits,60)
    while p==q : 
        q = RandomPrime(size_bits,60)
    N = p*q
    phi = (p-1) * (q-1)
    gcd,_,_ = xgcd(N,phi)  
    while gcd!=1:
        p = RandomPrime(size_bits,60)
        q = RandomPrime(size_bits,60)
        while p==q : 
            q = RandomPrime(size_bits,60)
        N = p*q
        phi = (p-1) * (q-1)
        gcd,_,_ = xgcd(N,phi) 
        
    l = LCM((q-1),(p-1))
    g = secrets.randbelow(N*N)
    try : 
            
        mu = InverseMod(L(pow(g,l,N*N),N),N)
    except ValueError :
        create_key_paire(size_bits)
        
    return PublicKey(g,N,N*N),PrivateKey(l,mu)

In [244]:
public_key,private_key = create_key_paire()

In [245]:
g,N, N_sqr = public_key

In [312]:
def encrypt(public_key : PublicKey, plain_text:int)-> int: 
    g,N, N_sqr = public_key
    r = secrets.randbelow(N)
    gcd,_,_ = xgcd(r,N)
    while gcd !=1:
        r = secrets.randbelow(N)
        gcd,_,_ = xgcd(r,N)


    cipher_text = (pow(g,plain_text,N_sqr) * pow(r,N,N_sqr))% (N_sqr)

    return cipher_text

def decrypt(public_key : PublicKey, private_key : PrivateKey, cipher_text : int)-> int: 
    l , mu = private_key
    _,N, N_sqr = public_key
    recovred_message = (L(pow(cipher_text,l,N_sqr),N)* mu) % N
    return recovred_message

def add(public_key : PublicKey , cipher_text_1 : int, cipher_text_2 : int)-> int: 
    return (cipher_text_1 * cipher_text_2 ) %public_key.n_sqr
def multiply(public_key : PublicKey , cipher_text_1 : int, plain_text_2 : int)-> int: 
    #we have 2 cases : 
    if plain_text_2==0:
        #if the plain_text_2 is 0 so everything multiplying by 0 is 0 so we encrypt the 0
        return encrypt(public_key,0)
    if plain_text_2==1:
        #if the plain_text_2 is 1 so everything multiplying by 1 it stays like ti is so it's equivalent of adding 0
        encryted_0 = encrypt(public_key,0)
        return add(public_key,cipher_text_1,encryted_0)
    return pow(cipher_text_1,plain_text_2,public_key.n_sqr)


# Test section 

here we define some fonctions to test the encrypt,decrypt,multiply and adding fonctions defined above

In [317]:
TEST_BIT_LENGTH = 32


def test_encrypt_and_decrypt():
    public_key,private_key  = create_key_paire(TEST_BIT_LENGTH)
    plaintext = 123

    ciphertext = encrypt(public_key, plaintext)
    assert ciphertext != plaintext

    decrypted = decrypt(public_key,private_key, ciphertext)
    assert decrypted == plaintext


def test_add():
    public_key, private_key = create_key_paire(TEST_BIT_LENGTH)
    a = 123
    b = 37
    expected = (123 + 37) % public_key.n

    ciphertext_a = encrypt(public_key, a)
    ciphertext_b = encrypt(public_key, b)

    encrypted_result = add(public_key, ciphertext_a, ciphertext_b)
    result = decrypt(public_key,private_key, encrypted_result)

    assert result == expected


def test_multiply():
    public_key, private_key = create_key_paire(TEST_BIT_LENGTH)
    a = 123
    b = 25
    expected = (123 * 25) % public_key.n

    ciphertext_a = encrypt(public_key, a)
    encrypted_result = multiply(public_key, ciphertext_a, b)
    result = decrypt(public_key,private_key, encrypted_result)

    assert result == expected


def test_multiply_by_zero():
    public_key, private_key = create_key_paire(TEST_BIT_LENGTH)
    a = 123
    b = 0
    expected = 0
    naive_encrypted_result = 1
    ciphertext_a = encrypt(public_key, a)

    encrypted_result = multiply(public_key, ciphertext_a, b)
    assert encrypted_result != naive_encrypted_result

    result = decrypt(public_key,private_key,  encrypted_result)
    assert result == expected


def test_multiply_by_one():
    public_key, private_key = create_key_paire(TEST_BIT_LENGTH)
    a = 123
    b = 1
    expected = 123
    ciphertext_a = encrypt(public_key, a)
    naive_encrypted_result = ciphertext_a

    encrypted_result = multiply(public_key, ciphertext_a, b)
    assert encrypted_result != naive_encrypted_result

    result = decrypt(public_key, private_key,  encrypted_result)
    assert result == expected


In [370]:
test_encrypt_and_decrypt()

In [371]:
test_add()

In [372]:
test_multiply()

In [373]:
test_multiply_by_zero()

In [374]:
test_multiply_by_one()

# Using POO to implement the Paillier Cryptosystem

In [388]:
class Paillier(): 
    def __init__(self,size_bits:int = DEFAULT_BIT_SIZE): 
        self.bit_size = size_bits
        p = RandomPrime(size_bits,60)
        q = RandomPrime(size_bits,60)
        while p==q : 
            q = RandomPrime(size_bits,60)
        N = p*q
        phi = (p-1) * (q-1)
        gcd,_,_ = xgcd(N,phi)  
        while gcd!=1:
            p = RandomPrime(size_bits,60)
            q = RandomPrime(size_bits,60)
            while p==q : 
                q = RandomPrime(size_bits,60)
            N = p*q
            phi = (p-1) * (q-1)
            gcd,_,_ = xgcd(N,phi) 

        l = LCM((q-1),(p-1))
        g = secrets.randbelow(N*N) 
        mu = InverseMod(L(pow(g,l,N*N),N),N)

        self.public_key = PublicKey(g,N,N*N)
        self.private_key = PrivateKey(l,mu)
    def encrypt(self,plain_text): 
        g,N, N_sqr = self.public_key
        r = secrets.randbelow(N)
        gcd,_,_ = xgcd(r,N)
        while gcd !=1:
            r = secrets.randbelow(N)
            gcd,_,_ = xgcd(r,N)


        cipher_text = (pow(g,plain_text,N_sqr) * pow(r,N,N_sqr))% (N_sqr)

        return cipher_text
    def decrypt(self,cipher_text): 
        l , mu = self.private_key
        _,N, N_sqr = self.public_key
        recovred_message = (L(pow(cipher_text,l,N_sqr),N)* mu) % N
        return recovred_message
    def add(self,cipher_text_1 : int, cipher_text_2 : int)-> int: 
        return (cipher_text_1 * cipher_text_2 ) % self.public_key.n_sqr
    
    def multiply(self,cipher_text_1 : int, plain_text_2 : int)-> int: 
        #we have 2 cases : 
        if plain_text_2==0:
            #if the plain_text_2 is 0 so everything multiplying by 0 is 0 so we encrypt the 0
            return encrypt(self.public_key,0)
        if plain_text_2==1:
            #if the plain_text_2 is 1 so everything multiplying by 1 it stays like ti is so it's equivalent of adding 0
            encryted_0 = encrypt(self.public_key,0)
            return add(self.public_key,cipher_text_1,encryted_0)
        return pow(cipher_text_1,plain_text_2,self.public_key.n_sqr)
    def __str__(self):
        output = f"""The Paillier parameters:\nPublic key :\n\tg:{self.public_key.g}\n\tn:{self.public_key.n}
        \nPrivate key:\n\tmu:{self.private_key.mu}\n\tlamda: {self.private_key.lam}"""
        return output

In [389]:
paillier1 = Paillier(DEFAULT_BIT_SIZE)

In [390]:
message = b'Hey my name is Paillier'
message_int = int.from_bytes(message,"big")
cipher_text = paillier1.encrypt(message_int)
print(f"The cipher text is :\n\t{cipher_text}")
recovred_text_int = paillier1.decrypt(cipher_text)
print(f"The plain text (int version) : \n\t{recovred_text_int}")
# recovred_text = recovred_text_int.to_bytes(len(message),"big")
# print(f"The plain text : \n\t{recovred_text}")

The cipher text is :
	6125048113372699287355980053430259525862683012433709381686023418968284373046
The plain text (int version) : 
	41596242906461611250364749554132564195


In [391]:
print(paillier1)

The Paillier parameters:
Public key :
	g:3727595063861091738273622875640156644160196046509090120382575895129117582432
	n:106669509487093997695988628296606263037
        
Private key:
	mu:28515140391437081981025493157355902032
	lamda: 53334754743546998837662089373891779630


In [398]:
a = 123
b = 37
expected = (123 + 37) % paillier1.public_key.n_sqr

ciphertext_a = paillier1.encrypt( a)
ciphertext_b = paillier1.encrypt(b)

encrypted_result = paillier1.add(ciphertext_a, ciphertext_b)
result = paillier1.decrypt(encrypted_result)
print(result)
print("\n")
print(expected)
assert result == expected


160


160


In [399]:
a = 123
b = 25
expected = (123 * 25) % paillier1.public_key.n

ciphertext_a = paillier1.encrypt( a)
encrypted_result = paillier1.multiply( ciphertext_a, b)
result = paillier1.decrypt( encrypted_result)

assert result == expected